In [1]:
import pandas as pd
import numpy as np
import os
from analysis import *
import glob
import string
import plotly
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
colors = ['#e6194B', '#3cb44b', '#ffe119','#f58231','#42d4f4',
          '#f032e6','#fabed4','#469990','#aaffc3','#000075','#e6194B','#9a6324','#dcbeff']
plotted_models = ['DPN','ReXNet', 'EseVoVNet', 'TNT', 'Inception', 'HRNet', 'MobileNet', 'VGG19', 'ResNet-RS', 
                 'DenseNet', 'DPN_Magface', 'DPN_Cosface', 'SMAC']

In [3]:
df_rank_4 = pd.read_csv("CelebA_val.csv")
df_rank_4_test = pd.read_csv("CelebA_test.csv")

In [19]:
#Test plot
fig = px.scatter(df_rank_4_test, 
                 x='Accuracy_mean', 
                 y='Disparity_mean', 
                 error_x = "Accuracy_std", 
                 error_y = "Disparity_std", 
                 color="Model", 
                 color_discrete_sequence=df_rank_4_test['color'],
                 template="simple_white",
                 width=1200, height= 1000
                )
fig.update_layout(
    yaxis_title="Rank Disparity",
    xaxis_title = "Error",
    xaxis_range=[0,0.4],
    yaxis_range=[0,10]
)
fig.update_layout(
    xaxis_title="Error",
    yaxis_title="Rank Disparity",
    legend_title="Models",
    font=dict(
        family="Times New Roman",
        size=38,
        color="Black"
    )
)
fig.update_traces(marker=dict(size=20))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
    font=dict(
        family="Times New Roman",
        size=28,
        color="Black"
    )
))
p = np.array(preparePareto(df_rank_4_test[['Accuracy_mean','Disparity_mean']], False, False).dropna())
for x, y in zip(p[:-1], p[1:]):
    fig.add_shape(type='line',
                x0=x[0],y0=x[1],x1=y[0],y1=y[1],
                line=dict(color='gray',width=4),line_dash='dash',
                xref='x',yref='y')
fig.update_layout(showlegend=False)
plotly.io.write_image(fig, '_RQ2_test.png', format='png')
fig.show()

In [18]:
#Test plot
fig = px.scatter(df_rank_4, 
                 x='Accuracy_mean', 
                 y='Disparity_mean', 
                 error_x = "Accuracy_std", 
                 error_y = "Disparity_std", 
                 color="Model", 
                 color_discrete_sequence=df_rank_4['color'],
                 template="simple_white",
                 width=1200, height= 1000
                )
fig.update_layout(
    yaxis_title="Rank Disparity",
    xaxis_title = "Error",
    xaxis_range=[0,0.4],
    yaxis_range=[0,10]
)
fig.update_layout(
    xaxis_title="Error",
    yaxis_title="Rank Disparity",
    legend_title="Models",
    font=dict(
        family="Times New Roman",
        size=38,
        color="Black"
    )
)
fig.update_traces(marker=dict(size=20))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
    font=dict(
        family="Times New Roman",
        size=28,
        color="Black"
    )
))
p = np.array(preparePareto(df_rank_4[['Accuracy_mean','Disparity_mean']], False, False).dropna())
for x, y in zip(p[:-1], p[1:]):
    fig.add_shape(type='line',
                x0=x[0],y0=x[1],x1=y[0],y1=y[1],
                line=dict(color='gray',width=4),line_dash='dash',
                xref='x',yref='y')
fig.update_layout(showlegend=False)
plotly.io.write_image(fig, '_RQ2.png', format='png')
fig.show()